In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
import random

In [2]:
# This parameter identifies the class of the table to be downloaded
#set_table_class = 'tblNormal'
set_table_class = 'wikitable'

# Option 1: This part generates the list of links to the webpages (use second line in case of one link)
links_data = ["https://simpsons.fandom.com/wiki/Season_{}".format(i) for i in range(1, 3+1)]
#links_data = ["https://simpsons.fandom.com/wiki/Season_1"]

# Option 2: This parameter sets the file with the list of the links to the webpages
#set_links_list = 'links_wc.csv'

# Output CSV file name
output_csv = 'simps_table.csv'

# Output XLSX file name
#output_xlsx = 'simps.xlsx'

In [3]:
# TODO: further error handling
# TODO: rowspan also
# TODO: more tables with the same class

def download_table(link, table_class):
    
    #### Returns the downloaded table ###
    
    # Downloading and parsing the site
    
    page = requests.get(link)
    soup = bs(page.text, 'html.parser')
    
    soup_table = soup.find_all('table', {'class': table_class})
    columns = []
    if len(soup_table) == 1:

        # Getting the column headers
        for th_inst in soup_table[0].find_all('th'):
            columns.append(th_inst.text.replace('\r\n', '').replace(' ', ''))
            columns.append(th_inst.text.replace('\r\n', '').replace(' ', '') + '_links')
            columns.append(th_inst.text.replace('\r\n', '').replace(' ', '') + '_images')
            if 'colspan' in th_inst.attrs:
                for i in range(int(th_inst.attrs['colspan'])-1):
                    columns.append(th_inst.text.replace('\r\n', '').replace(' ', '') + "_" + str(i))
                    columns.append(th_inst.text.replace('\r\n', '').replace(' ', '') + "_" + str(i) + '_links')
                    columns.append(th_inst.text.replace('\r\n', '').replace(' ', '') + "_" + str(i) + '_images')

        # Getting the data
        rows = []
        for tr_inst in soup_table[0].find_all('tr'):
            row = []
            for td_inst in tr_inst.find_all('td'):
                # This row gives an opportunity to condense the text, you can switch the comments for the next two lines
                #row = row + [td_inst.text.replace('\r\n', '').replace(' ', '')]
                row = row + [td_inst.text]
                # Getting all the hiperlinks
                links = td_inst.find_all('a')
                if len(links) > 0:
                    row = row + [' '.join(link['href'] for link in links)]
                else:
                    row = row + [None]
                # Getting all the images ...
                imgs = td_inst.find_all('img')
                if len(imgs) > 0:
                    row = row + [' '.join(img['src'] for img in imgs)]
                else:
                    row = row + [None]
            rows = rows + [row]

        # Building the dataframe with the columns and data, returning it
        print (link + " downloaded.")
        return pd.DataFrame(rows, columns = columns)

    else:
        print("Problemo. The number of tables found based on the given parameter: " + str(len(soup_table)))

In [4]:
# Getting the list of links

# Option 1: using the generated list
df_links = pd.DataFrame(links_data, columns = ['Links'])

# Option 2: loading a csv
#df_links = pd.read_csv(set_links_list, header=None)
#df_links.columns = ['Links']

df_links.head()

,Links
0,https://simpsons.fandom.com/wiki/Season_1
1,https://simpsons.fandom.com/wiki/Season_2
2,https://simpsons.fandom.com/wiki/Season_3


In [5]:
df_big_table = pd.DataFrame()

for link in df_links['Links']:
    df_big_table = df_big_table.append(download_table(link, set_table_class), ignore_index = True)
    time.sleep(random.uniform(1,2))
print("All done.")

https://simpsons.fandom.com/wiki/Season_1 downloaded.
https://simpsons.fandom.com/wiki/Season_2 downloaded.
https://simpsons.fandom.com/wiki/Season_3 downloaded.
All done.


In [6]:
df_big_table.head()

,#,# _images,# _links,Directedby,Directedby _images,Directedby _links,Originalairdate,Originalairdate _images,Originalairdate _links,Originaltitle(top)Alternatetitle(bottom),...,Prod.code _links,Title,Title _images,Title _links,TitleReference,TitleReference _images,TitleReference _links,Writtenby,Writtenby _images,Writtenby _links
0,None,None,None,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,None,None,None,None,None,None
1,1 - 1\n,None,None,David Silverman\n,None,/wiki/David_Silverman,"December 17, 1989\n",None,None,"\n""Simpsons Roasting on an Open Fire""\n""The Si...",...,None,NaN,NaN,NaN,Chestnuts Roasting on an Open Fire\n,None,None,Mimi Pond\n,None,/wiki/Mimi_Pond
2,None,None,None,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,None,None,None,None,None,None
3,2 - 2\n,None,None,David Silverman\n,None,/wiki/David_Silverman,"January 14, 1990\n",None,None,"""Bart the Genius""\n",...,None,NaN,NaN,NaN,N/A\n,None,None,Jon Vitti\n,None,/wiki/Jon_Vitti
4,None,None,None,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,None,None,None,None,None,None


In [7]:
# Removing all empty columns and rows

df_big_table = df_big_table.dropna(how='all', axis='columns')
df_big_table = df_big_table.dropna(how='all', axis='index')

In [8]:
# Writing results out to csv or xslx

df_big_table.to_csv(output_csv, index = False)
#df_big_table.to_excel(output_xlsx, index = False)